In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")
sys.path.insert(0, "../")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import utils
import time
import os

from unicodedata import category
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

import WOSutilities as wosutil

path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
# we need to load articles and categories
article_df = wosutil.load_wos_data(name = 'article', 
                                   path2rawdata = path2rawdata,
                                   year_list = None, 
                                   columns = ['ArticleID', 'PubYear', 'Doctypes', 'Title','Journal', 'Journal Abbreviation ISO'], 
                                   dropna = ['ArticleID', 'PubYear','Title'], 
                                   duplicate_subset = ['ArticleID'],
                                   isindict = {'Doctypes':np.sort(['Article','Letter','Review','Note'])}, 
                                   verbose = 50)

50
100
150
200
250
300
350
400
450
Final DF Shape (39171585, 6)


In [5]:
article_df.columns

Index(['ArticleID', 'PubYear', 'Doctypes', 'Title', 'Journal',
       'Journal Abbreviation ISO'],
      dtype='object')

In [6]:
article_info = article_df[(article_df.Journal == "NATURE") |
                           (article_df.Journal == "SCIENCE")]

article_info.groupby("Journal").size()

Journal
NATURE     160574
SCIENCE     89830
dtype: int64

In [7]:
# join with citations data
impact_df_path = "/mnt/disks/vault/analysis-data/impact-c5-data/impact_data_until_2010.pql"
impact_df = pd.read_pickle(impact_df_path)

data = None
data = article_info.merge(impact_df, 
                          how = 'inner',
                          on = 'ArticleID')

In [8]:
data.groupby("Journal").size()

Journal
NATURE     153697
SCIENCE     83598
dtype: int64

In [9]:
data.to_pickle("/mnt/disks/vault/analysis-data/nature-science-data-full/nature_science_journal_data.pql")

In [15]:
foo = list(data["PubYear_x"].unique())
foo.sort()

In [16]:
data.head(10)

,ArticleID,PubYear_x,Doctypes,Title,Journal,Journal Abbreviation ISO,PubYear_y,c5
0,A1981MB41800038,1981,Article,INDEPENDENT CONTROL OF IMMUNOGLOBULIN HEAVY AN...,NATURE,Nature,1981,23.0
1,A1981MK47400052,1981,Article,DEMETHYLATION OF CPG SITES IN DNA OF EARLY RAB...,NATURE,Nature,1981,14.0
2,A1981LV58600029,1981,Article,EPIDERMAL GROWTH-FACTOR ENHANCES VIRAL TRANSFO...,SCIENCE,Science,1981,22.0
3,A1981MT15400002,1981,Letter,THE PEER-REVIEW QUESTION,SCIENCE,Science,1981,0.0
4,A1981MD87000011,1981,Article,GENETIC-EFFECTS OF THE ATOMIC BOMBS - A REAPPR...,SCIENCE,Science,1981,58.0
5,000204440900038,1981,Article,Comb-layering in carbonatite dykes,NATURE,Nature,1981,5.0
6,A1981MJ81600013,1981,Article,RECENT DEVELOPMENTS IN NUCLEAR MAGNETIC-RESONA...,SCIENCE,Science,1981,7.0
7,A1981LU76700015,1981,Article,PRECAMBRIAN PERSPECTIVES,SCIENCE,Science,1981,21.0
8,A1981MT99400035,1981,Article,STRUCTURE SENSITIVITY IN THE IRON SINGLE-CRYST...,NATURE,Nature,1981,6.0
9,A1981LS25500015,1981,Article,IR VARIABILITY OF SS433,NATURE,Nature,1981,4.0
